In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from  tensorflow.keras.layers import Input, Dense
from  tensorflow.keras.models import Model
plt.rcParams['figure.figsize'] = [20, 8]

In [2]:
import time
import psutil
import os

In [3]:
data=pd.read_csv('C:\\Users\\vaish\\OneDrive\\Desktop\\hpe\\data_new.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Admin/OneDrive/Desktop/HPE/data_new.csv'

In [4]:
data_b=data.iloc[:,0:-3:2]

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_df = scaler.fit_transform(data_b)
df = pd.DataFrame(data_df, columns=data_b.columns,index=data.index)

In [6]:
input_dim = 30 # number of features in the input data
encoding_dim = 10 # number of neurons in the bottleneck layer
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

In [7]:
autoencoder.compile(optimizer='adam', loss='mse')

In [8]:
start_wall_time = time.time()
start_cpu_time = time.process_time()
autoencoder.fit(df, df, epochs=100, batch_size=32)

Epoch 1/100
32/32 [==============================] - 2s 3ms/step - loss: 0.0503
Epoch 2/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0455
Epoch 3/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0417
Epoch 4/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0373
Epoch 5/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0332
Epoch 6/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0296
Epoch 7/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0268
Epoch 8/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0245
Epoch 9/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0227
Epoch 10/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0213
Epoch 11/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0201
Epoch 12/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0191
Epoch 13/100
32/32 [=================

In [9]:
reconstructed_data = autoencoder.predict(df)
# get the end time
end_wall_time = time.time()
end_cpu_time = time.process_time()

mse = np.mean(np.power(df - reconstructed_data, 2), axis=1)
threshold = np.percentile(mse, 95)


32/32 [==============================] - 0s 2ms/step


In [10]:
anomalies=pd.DataFrame(index=mse.index)
anomalies['prediction']=np.where(mse>threshold,1,0)



In [115]:
known_df=data.iloc[:,-1]

In [11]:
# get the execution time
wall_time = end_wall_time - start_wall_time
cpu_time = end_cpu_time - start_cpu_time

In [12]:
print(f'Elapsed time: {int(wall_time)} seconds')
print(f'CPU time: {int(cpu_time)} seconds')

Elapsed time: 26 seconds
CPU time: 4 seconds


In [13]:
print(f'Physical Memory usage: {int(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)} MB') 
print(f'Virtual Memory usage:  {int(psutil.Process(os.getpid()).memory_info().vms / 1024 ** 2)} MB')

Physical Memory usage: 380 MB
Virtual Memory usage:  339 MB


In [14]:
anomalies['prediction']

timestamp
2021-01-01 00:00:00    1
2021-01-01 01:00:00    1
2021-01-01 02:00:00    1
2021-01-01 03:00:00    1
2021-01-01 04:00:00    1
                      ..
2021-02-11 19:00:00    0
2021-02-11 20:00:00    1
2021-02-11 21:00:00    0
2021-02-11 22:00:00    0
2021-02-11 23:00:00    0
Name: prediction, Length: 1008, dtype: int32

In [15]:
#PERFORMANCE PREDICTION
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
import pandas as pd


true_values = data['Overall anomaly'].values
predicted_values = anomalies['prediction'].values
precision = precision_score(true_values, predicted_values)
f1=f1_score(true_values, predicted_values)
recall=recall_score(true_values, predicted_values)
accuracy= accuracy_score(true_values, predicted_values)

print("precision=",precision)
print("f1=",f1)
print("recall=",recall)
print("accuracy=",accuracy)

precision= 0.3333333333333333
f1= 0.10828025477707005
recall= 0.06463878326996197
accuracy= 0.7222222222222222
